In [1]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=True)

In [4]:
base_path = os.path.abspath('../../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
sys.path.append(strategy_path)

In [5]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [6]:
from EMDY import EMDY
from EMPE import EMPE
from EMPM import EMPM
from EMSS import EMSS
from EMCA import EMCA

In [7]:
# EM equity data
emindex = pd.read_csv(os.path.join(data_path, 'totindex-em.csv'), header=0, index_col=0, parse_dates=True)
emindex1 = pd.read_csv(os.path.join(data_path, 'priceindex-em.csv'), header=0, index_col=0, parse_dates=True)
emfut = pd.read_csv(os.path.join(data_path, 'fut1return-em.CSV'), header=0, index_col=0, parse_dates=True)

total_em_ret = emindex.pct_change()
prc_em_ret = emindex1.pct_change()
fut_em_ret = emfut.pct_change()
fut_em_ret = fut_em_ret.reindex(pd.date_range(fut_em_ret.index[0], fut_em_ret.index[-1], freq ='B')) # bizday로 변환

prc_em_ret.loc[total_em_ret.index] = total_em_ret
raw_index = pd.concat([emindex1.loc[:'2012-12-31'], emfut.loc['2013-01-01':]]).iloc[1:]
EMRet = pd.concat([prc_em_ret.loc[:'2012-12-31'], fut_em_ret.loc['2013-01-01':]]).iloc[1:]

EMRet.fillna(0, inplace = True)
EMindex = EMRet
EMindex.iloc[0] = 0
EMindex = (1+ EMindex).cumprod()
EMindex = EMindex / EMindex.iloc[0]

raw_index.index.name = 'tdate'
raw_index.columns.name = 'ticker'

EMindex.index.name = 'tdate'
EMRet.index.name = 'tdate'

EMindex.columns.name = 'ticker'
EMRet.columns.name = 'ticker'

raw_index = pd.read_csv(os.path.join(data_path,'priceindex-mon-em.csv') ,header=0, index_col=0, parse_dates=True)
raw_index.index.name = 'tdate'
raw_index.columns.name = 'ticker'

# 1. EMSS

In [58]:
emss = EMSS(strategy_name="EMSS", asset_type="EMERGING")
emss.load_index_and_return(from_db=True, save_file=True)
emss.set_rebalance_period(ts_freq='month', cs_freq='month')  # rebalance_day: monday = 0, sunday = 6

2020-01-21 10:53:04,358 - EMSS - INFO - [STEP 0] START LOGGING EMSS
2020-01-21 10:53:04,358 - EMSS - INFO - [STEP 0] START LOGGING EMSS
2020-01-21 10:53:04,358 - EMSS - INFO - [STEP 0] START LOGGING EMSS
2020-01-21 10:53:04,358 - EMSS - INFO - [STEP 0] START LOGGING EMSS
2020-01-21 10:53:04,360 - EMSS - INFO - [STEP 1] LOAD DATA
2020-01-21 10:53:04,360 - EMSS - INFO - [STEP 1] LOAD DATA
2020-01-21 10:53:04,360 - EMSS - INFO - [STEP 1] LOAD DATA
2020-01-21 10:53:04,360 - EMSS - INFO - [STEP 1] LOAD DATA
2020-01-21 10:53:04,361 - EMSS - INFO - [STEP 1 - 1] CONNECT TO BLOOM DATABASE
2020-01-21 10:53:04,361 - EMSS - INFO - [STEP 1 - 1] CONNECT TO BLOOM DATABASE
2020-01-21 10:53:04,361 - EMSS - INFO - [STEP 1 - 1] CONNECT TO BLOOM DATABASE
2020-01-21 10:53:04,361 - EMSS - INFO - [STEP 1 - 1] CONNECT TO BLOOM DATABASE
2020-01-21 10:53:04,363 - EMSS - INFO - [STEP 1 - 3] CHANGE BLOOM AND PAST DATA INTO STANDARD FORMAT
2020-01-21 10:53:04,363 - EMSS - INFO - [STEP 1 - 3] CHANGE BLOOM AND PAST 

In [59]:
# emss.index = EMindex
# emss.ret = EMRet

In [60]:
CS=0.35
short=0.2
day1=24
fundwgt=1
statwgt=1

In [61]:
group_1 = ['CN', 'KR', 'TW', 'IN']
group_2 = ['MY', 'BR', 'MX', 'RU', 'SA']

for type in range(2):
    if type == 0:
        RET = emss.ret.loc[:, group_1]
        index = emss.index.loc[:, RET.columns]
    else:
        RET = emss.ret.loc[:, group_2]
        index = emss.index.loc[:, RET.columns]
    Ret = RET

    TOM = (RET.index + pd.tseries.offsets.BDay(1)).day
    TOM2 = pd.DataFrame([-short] * len(TOM), index=Ret.index)
    TOM2[TOM >= day1] = 1
    SIG = Ret * 0 + 1
    SIG = pd.DataFrame(SIG.values * TOM2.values, index=TOM2.index, columns=Ret.columns)
    TSRV1 = SIG * fundwgt

    short = 0
    CSRV = index * 0

    statday = (Ret.index + pd.tseries.offsets.DateOffset(months=1)).strftime('%Y-%m').unique()
    bible_idx = statday[36 - short:len(statday) - short]
    bible_ls = []

    for i in range(len(bible_idx)):
        ave = np.mean(Ret[(Ret.index < bible_idx[i]) & (Ret.index >= statday[i]) & (
                    Ret.index.month != pd.to_datetime(bible_idx[i]).month)], axis=0)
        ave = ave.iloc[:len(index.columns)]
        bible_temp = np.mean(Ret[(Ret.index < bible_idx[i]) & (Ret.index >= statday[i]) & (
                    Ret.index.month == pd.to_datetime(bible_idx[i]).month)], axis=0) / \
                     np.std(Ret[(Ret.index < bible_idx[i]) & (Ret.index >= statday[i]) & (
                                 Ret.index.month == pd.to_datetime(bible_idx[i]).month)], axis=0)
        bible_ls.append(bible_temp.iloc[:len(index.columns)])

    bible = pd.DataFrame(bible_ls, index=bible_idx)
    RV1 = bible.iloc[:, :len(index.columns)]

    truecount = np.round(RV1.notnull().sum(axis=1)*CS)
    truecount = np.transpose(np.tile(truecount, (len(RV1.columns), 1)))

    bibleRV = RV1.rank(axis=1, method='first')
    bibleRV1 = -1* bibleRV.sub(RV1.count(axis=1), axis=0) +1
    bibleRVpos = bibleRV * 0
    bibleRVpos[bibleRV <= truecount] = -1
    bibleRVpos[bibleRV1 <= truecount] = 1

    for i in range(len(bible)):
        CSRV_ym = (pd.to_datetime(bible_idx[i]) - pd.tseries.offsets.DateOffset(months=1)).strftime('%Y-%m')
        CSRV_range = CSRV[CSRV.index.strftime('%Y-%m') == CSRV_ym]
        CSRV[CSRV.index.strftime('%Y-%m') == CSRV_ym] = np.tile(bibleRVpos.iloc[i].values * statwgt, (CSRV_range.shape[0], 1))

    bibleTS1 = bible
    bibleTS = bibleTS1 * 0
    bibleTS[bibleTS1 < (-0.5)] = -1
    bibleTS[bibleTS1 > (0.5)] = 1
    TSRV = CSRV * 0

    for i in range(len(bible)):
        TSRV_ym = (pd.to_datetime(bible_idx[i])).strftime('%Y-%m')
        TSRV_range = TSRV[TSRV.index.strftime('%Y-%m') == TSRV_ym]
        TSRV[TSRV.index.strftime('%Y-%m') == TSRV_ym] = [bibleTS.iloc[i, :]] * TSRV_range.shape[0]

    AAA = [x for x in TSRV1 if x in TSRV]
    TSRV1 = TSRV1.loc[:, AAA]
    TSRV = TSRV.loc[:, AAA]

    TSRV = TSRV1 * 1 + TSRV * 0

    if type == 0:
        TSRVrun1 = TSRV
        CSRVrun1 = CSRV
    else:
        TSRVrun2 = TSRV
        CSRVrun2 = CSRV

In [62]:
TSRV_ym

'2019-12'

In [63]:
TSRV.tail()

ticker,MY,BR,MX,RU,SA
tdate,,,,,
2019-11-19,0.0,0.0,0.0,0.0,0.0
2019-11-20,0.0,0.0,0.0,0.0,0.0
2019-11-21,0.0,0.0,0.0,0.0,0.0
2019-11-22,1.0,1.0,1.0,1.0,1.0
2019-11-25,1.0,1.0,1.0,1.0,1.0


In [64]:
CSRV.tail()

ticker,MY,BR,MX,RU,SA
tdate,,,,,
2019-11-19,1.0,-1.0,1.0,-1.0,0.0
2019-11-20,1.0,-1.0,1.0,-1.0,0.0
2019-11-21,1.0,-1.0,1.0,-1.0,0.0
2019-11-22,1.0,-1.0,1.0,-1.0,0.0
2019-11-25,1.0,-1.0,1.0,-1.0,0.0


In [25]:
bible_idx[-2]

'2019-11'

In [21]:
CSRV_ym

'2019-11'

In [15]:
TSRV_ym

'2019-12'

In [16]:
TSRV_range

ticker,MY,ID,BR,MX,RU,SA
tdate,,,,,,


In [17]:
TSRV.tail()

ticker,MY,ID,BR,MX,RU,SA
tdate,,,,,,
2019-11-19,-0.0,NaN,-0.0,-0.0,0.0,-0.0
2019-11-20,-0.0,NaN,-0.0,-0.0,0.0,-0.0
2019-11-21,-0.0,NaN,-0.0,-0.0,0.0,-0.0
2019-11-22,-0.0,NaN,-0.0,-0.0,0.0,-0.0
2019-11-25,-0.0,NaN,-0.0,-0.0,0.0,-0.0


In [ ]:

TSRV = pd.concat([TSRVrun1, TSRVrun2], axis=1)
CSRV = pd.concat([CSRVrun1, CSRVrun2], axis=1)

TSRV = TSRV[emss.ret.columns]
CSRV = CSRV[emss.ret.columns]

emss.TSRV = TSRV.loc[emss.ret.index].fillna(method='ffill').dropna(how='all').fillna(0) # add fillna(0) for 'ID'
emss.CSRV = CSRV.loc[emss.ret.index].fillna(method='ffill').dropna(how='all').fillna(0) # add fillna(0) for 'ID'

# Align dates with each other
if emss.TSRV.index[0] > emss.CSRV.index[0]:
    emss.CSRV = emss.CSRV.loc[emss.TSRV.index[0]:]
else:
    emss.TSRV = emss.TSRV.loc[emss.CSRV.index[0]:]

In [ ]:
emss.calculate_signal(CS=0.35, short=0.2, day1=24, fundwgt=1, statwgt=1)
emss.set_portfolio_parameter(cs_strategy_type="notional")
emss.make_portfolio()